In [1]:
import torch
import argparse
import numpy as np

from utils import *
from torch.utils.data import DataLoader
from solver import Solver
# from config import get_args, get_config, output_dim_dict, criterion_dict
from data_loader import get_loader
from test_instance import TestMOSI, TestMOSEI

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /home/soyeon/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /home/soyeon/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/soyeon/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f

In [2]:
def set_seed(seed):
    # torch.set_default_tensor_type('torch.FloatTensor')
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        # torch.set_default_tensor_type('torch.cuda.FloatTensor')

        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        use_cuda = True

In [3]:
# path to a pretrained word embedding file
word_emb_path = '/mnt/soyeon/workspace/glove.840B.300d.txt'
assert(word_emb_path is not None)

In [4]:
from datetime import datetime
from pathlib import Path
import pprint
from torch import optim
import torch.nn as nn

# username = Path.home().name
# project_dir = Path(__file__).resolve().parent.parent
# sdk_dir = project_dir.joinpath('CMU-MultimodalSDK')
# data_dir = project_dir.joinpath('datasets')
sdk_dir = Path('/mnt/soyeon/workspace/multimodal/CMU-MultimodalSDK')
data_dir = Path('/mnt/soyeon/workspace/multimodal/datasets')
data_dict = {'mosi': data_dir.joinpath('MOSI'), 'mosei': data_dir.joinpath(
    'MOSEI'), 'ur_funny': data_dir.joinpath('UR_FUNNY')}
optimizer_dict = {'RMSprop': optim.RMSprop, 'Adam': optim.Adam}
activation_dict = {'elu': nn.ELU, "hardshrink": nn.Hardshrink, "hardtanh": nn.Hardtanh,
                   "leakyrelu": nn.LeakyReLU, "prelu": nn.PReLU, "relu": nn.ReLU, "rrelu": nn.RReLU,
                   "tanh": nn.Tanh}

output_dim_dict = {
    'mosi': 1,
    'mosei_senti': 1,
}

criterion_dict = {
    'mosi': 'L1Loss',
    'iemocap': 'CrossEntropyLoss',
    'ur_funny': 'CrossEntropyLoss'
}

In [5]:
import easydict

args = easydict.EasyDict({
    # Tasks
    "dataset": "mosi",
    "data_path": "datasets",

    # Dropouts
    "dropout_a": 0.1,
    "dropout_v": 0.1,
    "dropout_prj": 0.1,

    # Architecture
    "multiseed": True,
    "contrast": True,
    "add_va": True,
    "n_layer": 1,
    "cpc_layers": 1,
    "d_vh": 16,
    "d_ah": 16,
    "d_vout": 16,
    "d_aout": 16,
    "bidirectional": True,
    "d_prjh": 128,
    "pretrain_emb": 768,

    # Activations
    "mmilb_mid_activation": "ReLU",
    "mmilb_last_activation": "Tanh",
    "cpc_activation": "Tanh",

    # Training Setting
    "batch_size": 32,
    "clip": 1.0,
    "lr_main": 1e-3,
    "lr_bert": 5e-5,
    "lr_mmilb": 1e-3,
    "alpha": 0.1,
    "beta": 0.1,
    "weight_decay_main": 1e-4,
    "weight_decay_bert": 1e-4,
    "weight_decay_club": 1e-4,
    "optim": "Adam",
    "num_epochs": 40,
    "when": 20,
    "patience": 10,
    "update_batch": 1,

    # Logistics
    "log_interval": 100,
    "seed": 1111
})

In [6]:
def str2bool(v):
    """string to boolean"""
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')

In [7]:
class Config(object):
    def __init__(self, data, mode='train'):
        """Configuration Class: set kwargs as class attributes with setattr"""
        self.dataset_dir = data_dict[data.lower()]
        self.sdk_dir = sdk_dir
        self.mode = mode
        # Glove path
        self.word_emb_path = word_emb_path

        # Data Split ex) 'train', 'valid', 'test'
        self.data_dir = self.dataset_dir

    def __str__(self):
        """Pretty-print configurations in alphabetical order"""
        config_str = 'Configurations\n'
        config_str += pprint.pformat(self.__dict__)
        return config_str


def get_config(dataset='mosi', mode='train', batch_size=32):
    config = Config(data=dataset, mode=mode)
    
    config.dataset = dataset
    config.batch_size = batch_size

    return config

In [8]:
dataset = str.lower(args.dataset.strip())

set_seed(args.seed)
print("Start loading the data....")
train_config = get_config(dataset, mode='train', batch_size=args.batch_size)
valid_config = get_config(dataset, mode='valid', batch_size=args.batch_size)
test_config = get_config(dataset, mode='test',  batch_size=args.batch_size)

# pretrained_emb saved in train_config here
train_loader = get_loader(args, train_config, shuffle=True)
print('Training data loaded!')
valid_loader = get_loader(args, valid_config, shuffle=False)
print('Validation data loaded!')
test_loader = get_loader(args, test_config, shuffle=False)
print('Test data loaded!')
print('Finish loading the data....')

torch.autograd.set_detect_anomaly(True)

# addintional appending
args.word2id = train_config.word2id

# architecture parameters
args.d_tin, args.d_vin, args.d_ain = train_config.tva_dim
args.dataset = args.data = dataset
args.when = args.when
args.n_class = output_dim_dict.get(dataset, 1)
args.criterion = criterion_dict.get(dataset, 'MSELoss')

Start loading the data....
train
Training data loaded!
valid
Validation data loaded!
test
Test data loaded!
Finish loading the data....


In [9]:
solver = Solver(args, train_loader=train_loader, dev_loader=valid_loader, test_loader=test_loader, is_train=True)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/soyeon/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/res

In [11]:
model = solver.model

In [12]:
model = solver.train_and_eval()

100%|██████████| 41/41 [00:14<00:00,  2.86it/s]


--------------------------------------------------
Epoch  1 | Time 28.3605 sec | Valid Loss 1.0112 | Test Loss 0.9232
--------------------------------------------------
MAE:  0.92319214
Correlation Coefficient:  0.7049294243292297
mult_acc_7:  0.29300291545189505
mult_acc_5:  0.3206997084548105
F1 score all/non0: 0.7791/0.8099 over 686/656
Accuracy all/non0: 0.7886/0.8171
--------------------------------------------------
Saved model at pre_trained_models/MM.pt!


100%|██████████| 41/41 [00:14<00:00,  2.82it/s]


--------------------------------------------------
Epoch  2 | Time 28.6376 sec | Valid Loss 0.8006 | Test Loss 0.9007
--------------------------------------------------
MAE:  0.9006584
Correlation Coefficient:  0.7722844640725065
mult_acc_7:  0.34839650145772594
mult_acc_5:  0.39504373177842567
F1 score all/non0: 0.8052/0.8121 over 686/656
Accuracy all/non0: 0.8047/0.811
--------------------------------------------------
Saved model at pre_trained_models/MM.pt!


100%|██████████| 41/41 [00:14<00:00,  2.86it/s]


...

--------------------------------------------------
Epoch 11 | Time 28.3694 sec | Valid Loss 0.6903 | Test Loss 0.7968
--------------------------------------------------


100%|██████████| 41/41 [00:14<00:00,  2.85it/s]


--------------------------------------------------
Epoch 12 | Time 28.4112 sec | Valid Loss 0.7238 | Test Loss 0.8273
--------------------------------------------------


 68%|██████▊   | 28/41 [00:09<00:04,  2.94it/s]


KeyboardInterrupt: 

In [11]:
torch.save(model.state_dict(), "./saved_models_MMIM_mosi.pt")

### Model Load

In [12]:
model.load_state_dict(torch.load("./saved_models_MMIM_mosi.pt"))
model.eval()

RuntimeError: Error(s) in loading state_dict for MMIM:
	Missing key(s) in state_dict: "visual_enc.rnn.weight_ih_l0_reverse", "visual_enc.rnn.weight_hh_l0_reverse", "visual_enc.rnn.bias_ih_l0_reverse", "visual_enc.rnn.bias_hh_l0_reverse", "acoustic_enc.rnn.weight_ih_l0_reverse", "acoustic_enc.rnn.weight_hh_l0_reverse", "acoustic_enc.rnn.bias_ih_l0_reverse", "acoustic_enc.rnn.bias_hh_l0_reverse", "mi_va.mlp_mu.0.weight", "mi_va.mlp_mu.0.bias", "mi_va.mlp_mu.2.weight", "mi_va.mlp_mu.2.bias", "mi_va.mlp_logvar.0.weight", "mi_va.mlp_logvar.0.bias", "mi_va.mlp_logvar.2.weight", "mi_va.mlp_logvar.2.bias", "mi_va.entropy_prj.0.weight", "mi_va.entropy_prj.0.bias". 
	size mismatch for visual_enc.linear_1.weight: copying a param with shape torch.Size([16, 16]) from checkpoint, the shape in current model is torch.Size([16, 32]).
	size mismatch for acoustic_enc.linear_1.weight: copying a param with shape torch.Size([16, 16]) from checkpoint, the shape in current model is torch.Size([16, 32]).